# Projet Data

## Nicolas, Georges et Michael
    

In [3]:
from pandas import read_excel
# Importing necessary library
import pandas as pd
import numpy as np
import nltk
import os
import nltk.corpus# sample text for performing tokenization
from lxml import html
import requests
from nltk.tokenize import word_tokenize# Passing the string text into word tokenize for breaking the sentences
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import FrenchStemmer
import unidecode
# faut lancer ca la premiere fois -> nltk.download()

## <font color = "darkgreen">1- Chargement des jeux de données</font>

In [4]:
df_data = pd.read_excel("Data/Table_Ciqual_2020_FR_2020_07_07.xls")

## <font color = "darkgreen">2- Analyse la bdd </font>

On va analyser la BDD pour mieux la connaître et par la suite mieux la nettoyer.

In [5]:
#On regarde toute les colonnes du DataFrame
df_data.columns

Index(['alim_grp_code', 'alim_ssgrp_code', 'alim_ssssgrp_code',
       'alim_grp_nom_fr', 'alim_ssgrp_nom_fr', 'alim_ssssgrp_nom_fr',
       'alim_code', 'alim_nom_fr', 'alim_nom_sci',
       'Energie, Règlement UE N° 1169/2011 (kJ/100 g)',
       'Energie, Règlement UE N° 1169/2011 (kcal/100 g)',
       'Energie, N x facteur Jones, avec fibres  (kJ/100 g)',
       'Energie, N x facteur Jones, avec fibres  (kcal/100 g)',
       'Eau (g/100 g)', 'Protéines, N x facteur de Jones (g/100 g)',
       'Protéines, N x 6.25 (g/100 g)', 'Glucides (g/100 g)',
       'Lipides (g/100 g)', 'Sucres (g/100 g)', 'Fructose (g/100 g)',
       'Galactose (g/100 g)', 'Glucose (g/100 g)', 'Lactose (g/100 g)',
       'Maltose (g/100 g)', 'Saccharose (g/100 g)', 'Amidon (g/100 g)',
       'Fibres alimentaires (g/100 g)', 'Polyols totaux (g/100 g)',
       'Cendres (g/100 g)', 'Alcool (g/100 g)', 'Acides organiques (g/100 g)',
       'AG saturés (g/100 g)', 'AG monoinsaturés (g/100 g)',
       'AG polyinsatur

In [6]:
#On garde que les colonne qui nous intéresse pour faire notre 'nutri-score'
df_propre = df_data.copy().drop(['alim_grp_code','alim_ssgrp_code','alim_ssssgrp_code','alim_ssssgrp_nom_fr','alim_ssgrp_code','alim_nom_sci', 
       'Energie, Règlement UE N° 1169/2011 (kJ/100 g)','Energie, N x facteur Jones, avec fibres  (kJ/100 g)',
       'Energie, N x facteur Jones, avec fibres  (kcal/100 g)',
       'Eau (g/100 g)', 'Protéines, N x facteur de Jones (g/100 g)', 'Fructose (g/100 g)',
       'Galactose (g/100 g)', 'Glucose (g/100 g)', 'Lactose (g/100 g)',
       'Maltose (g/100 g)', 'Saccharose (g/100 g)', 'Amidon (g/100 g)',
       'Fibres alimentaires (g/100 g)', 'Polyols totaux (g/100 g)',
       'Cendres (g/100 g)', 'Alcool (g/100 g)', 'Acides organiques (g/100 g)', 'AG monoinsaturés (g/100 g)',
       'AG polyinsaturés (g/100 g)', 'AG 4:0, butyrique (g/100 g)',
       'AG 6:0, caproïque (g/100 g)', 'AG 8:0, caprylique (g/100 g)',
       'AG 10:0, caprique (g/100 g)', 'AG 12:0, laurique (g/100 g)',
       'AG 14:0, myristique (g/100 g)', 'AG 16:0, palmitique (g/100 g)',
       'AG 18:0, stéarique (g/100 g)', 'AG 18:1 9c (n-9), oléique (g/100 g)',
       'AG 18:2 9c,12c (n-6), linoléique (g/100 g)',
       'AG 18:3 c9,c12,c15 (n-3), alpha-linolénique (g/100 g)',
       'AG 20:4 5c,8c,11c,14c (n-6), arachidonique (g/100 g)',
       'AG 20:5 5c,8c,11c,14c,17c (n-3) EPA (g/100 g)',
       'AG 22:6 4c,7c,10c,13c,16c,19c (n-3) DHA (g/100 g)',
       'Cholestérol (mg/100 g)', 'Sel chlorure de sodium (g/100 g)',
       'Calcium (mg/100 g)', 'Chlorure (mg/100 g)', 'Cuivre (mg/100 g)',
       'Fer (mg/100 g)', 'Iode (µg/100 g)', 'Magnésium (mg/100 g)',
       'Manganèse (mg/100 g)', 'Phosphore (mg/100 g)', 'Potassium (mg/100 g)',
       'Sélénium (µg/100 g)', 'Sodium (mg/100 g)', 'Zinc (mg/100 g)',
       'Rétinol (µg/100 g)', 'Beta-Carotène (µg/100 g)',
       'Vitamine D (µg/100 g)', 'Vitamine E (mg/100 g)',
       'Vitamine K1 (µg/100 g)', 'Vitamine K2 (µg/100 g)',
       'Vitamine C (mg/100 g)', 'Vitamine B1 ou Thiamine (mg/100 g)',
       'Vitamine B2 ou Riboflavine (mg/100 g)',
       'Vitamine B3 ou PP ou Niacine (mg/100 g)',
       'Vitamine B5 ou Acide pantothénique (mg/100 g)',
       'Vitamine B6 (mg/100 g)', 'Vitamine B9 ou Folates totaux (µg/100 g)',
       'Vitamine B12 (µg/100 g)'], axis = 1)

In [7]:
#On change les noms des colonnes pour qu'ils soient plus simple
df_propre.columns = ['alim_grp_nom_fr', 'alim_ssgrp_nom_fr', 'alim_code', 'alim_nom_fr', 'Kcal/100 g', 'Protéines/100 g', 'Glucides/100 g', 'Lipides/100 g' ,'Sucres/100 g','AG saturés/100 g']

In [8]:
#On regarde comment les aliments sont écrits dans la colonne pour venir les chercher dans le DataFrame
df_propre.loc[df_propre["alim_nom_fr"].apply(lambda x: "oeuf" in x)]

,alim_grp_nom_fr,alim_ssgrp_nom_fr,alim_code,alim_nom_fr,Kcal/100 g,Protéines/100 g,Glucides/100 g,Lipides/100 g,Sucres/100 g,AG saturés/100 g
42,entrées et plats composés,soupes,25930,"Bouillon de boeuf, déshydraté reconstitué","9,05","0,6","1,33","0,15","0,33","0,057"
72,entrées et plats composés,plats composés,25033,Boeuf bourguignon,"84,6","8,93","4,17","3,3","1,57","0,88"
81,entrées et plats composés,plats composés,25175,"Meloukhia, plat à base de boeuf et corete, fai...",189,"10,6","1,2","14,7","1,2",-
83,entrées et plats composés,plats composés,25201,"Langue de boeuf sauce madère, préemballée",180,"9,5","4,5","13,5","2,5",6
85,entrées et plats composés,plats composés,25211,"Boulettes au boeuf, à la sauce tomate, préemba...",117,9,"5,5","6,5",-,-
...,...,...,...,...,...,...,...,...,...,...
2511,eaux et autres boissons,boisson alcoolisées,1016,Marsala aux oeufs,153,0,"12,4",0,"12,4",0
2676,produits sucrés,biscuits sucrés,24430,Biscuit sec aux oeufs à la cuillère (cuiller) ...,-,"7,98","79,9","3,77","47,9","0,99"
2775,produits sucrés,gâteaux et pâtisseries,23481,"Tarte normande aux pommes (garniture farine, o...",313,"4,44","38,7",15,"25,8","8,37"
2785,produits sucrés,gâteaux et pâtisseries,23525,Flan pâtissier aux oeufs ou à la parisienne,-,"3,69","29,2","11,3","18,4","7,16"


In [9]:
#On regarde tous les groupes d'aliments qu'il y a
df_propre['alim_grp_nom_fr'].value_counts()

viandes, œufs, poissons et assimilés           788
fruits, légumes, légumineuses et oléagineux    614
entrées et plats composés                      337
produits sucrés                                306
produits laitiers et assimilés                 301
eaux et autres boissons                        295
aides culinaires et ingrédients divers         212
produits céréaliers                            145
matières grasses                                75
aliments infantiles                             39
glaces et sorbets                               29
Name: alim_grp_nom_fr, dtype: int64

In [10]:
#On regarde le nombre de 'nan' dans chaque colonne
df_propre.isnull().sum()

alim_grp_nom_fr      45
alim_ssgrp_nom_fr    45
alim_code             0
alim_nom_fr           0
Kcal/100 g           17
Protéines/100 g       1
Glucides/100 g        1
Lipides/100 g         1
Sucres/100 g          9
AG saturés/100 g      5
dtype: int64

In [11]:
#On print la somme des 'nan'
print(np.sum(df_propre.isnull().sum()))

124


In [12]:
#On remarque que 'nan' a 43 occurences du coup nous allons les changer pour traiter la colonne par la suite
df_propre['alim_ssgrp_nom_fr'].value_counts()

légumes                                             303
charcuteries et assimilés                           173
fruits                                              170
viandes crues                                       162
plats composés                                      159
                                                   ... 
desserts infantiles                                   5
huiles de poissons                                    4
céréales et biscuits infantiles                       4
-                                                     1
aides culinaires et ingrédients pour végétariens      1
Name: alim_ssgrp_nom_fr, Length: 64, dtype: int64

## <font color = "darkgreen">3- Nettoyage de la bdd </font>

### <font color = "blue">3.1- Valeur numérique </font>

On va nettoyer la bdd au niveau des valeurs numériques en supprimant les Nan, '-' et '<'.

In [13]:
#On va changer toutes les valeurs 'nan' en 'rien'
for column in df_propre['alim_ssgrp_nom_fr']:
            df_propre['alim_ssgrp_nom_fr'] = df_propre['alim_ssgrp_nom_fr'].replace(to_replace = np.nan, value = 'rien')

In [14]:
#alim_ssgrp_nom_fr est bien à 0
df_propre.isnull().sum()

alim_grp_nom_fr      45
alim_ssgrp_nom_fr     0
alim_code             0
alim_nom_fr           0
Kcal/100 g           17
Protéines/100 g       1
Glucides/100 g        1
Lipides/100 g         1
Sucres/100 g          9
AG saturés/100 g      5
dtype: int64

In [15]:
#On va changer toutes les valeurs '-' en 'rien' dans la colonne alim_ssgrp_nom_fr
for column in df_propre['alim_ssgrp_nom_fr']:
            df_propre['alim_ssgrp_nom_fr'] = df_propre['alim_ssgrp_nom_fr'].replace(['-'], ['rien'])

In [16]:
#On remarque qu'il y a plus de 'nan' et de '-', ils sont tous devenus 'rien'
df_propre['alim_ssgrp_nom_fr'].value_counts()

légumes                                              303
charcuteries et assimilés                            173
fruits                                               170
viandes crues                                        162
plats composés                                       159
                                                    ... 
denrées destinées à une alimentation particulière      5
desserts infantiles                                    5
huiles de poissons                                     4
céréales et biscuits infantiles                        4
aides culinaires et ingrédients pour végétariens       1
Name: alim_ssgrp_nom_fr, Length: 64, dtype: int64

In [17]:
#On supprime les alim_ssgrp_nom_fr qui nous servent pas
sous_groupe_sup = ['céréales et biscuits infantiles', 'rien', 'aides culinaires et ingrédients pour végétariens', 'huiles de poissons',
                  'desserts infantiles', 'denrées destinées à une alimentation particulière', 'aides culinaires et ingrédients pour végétariens',
                  'sorbets', 'barres céréalières', 'glaces', 'desserts glacés', 'petits pots salés et plats infantiles',
                  'autres produits à base de viande', 'laits et boissons infantiles', 'confiseries non chocolatées',
                  'biscuits apéritifs', 'salades composées et crudités', 'viennoiseries', 'feuilletées et autres entrées',
                  'sandwichs', 'pizzas, tartes et crêpes salées', 'gâteaux et pâtisseries', 'plats composés']
i = 0

while(i < len(sous_groupe_sup)):
    df_propre.drop(df_propre[df_propre['alim_ssgrp_nom_fr'] == sous_groupe_sup[i]].index, inplace = True)
    i = i + 1


In [18]:
#On vérifie bien que les alim_ssgrp_nom_fr sont supprimées
df_propre['alim_ssgrp_nom_fr'].value_counts()

légumes                                              303
charcuteries et assimilés                            173
fruits                                               170
viandes crues                                        162
boissons sans alcool                                 156
produits laitiers frais et assimilés                 137
fromages et assimilés                                135
viandes cuites                                       133
poissons crus                                        106
eaux                                                  90
sauces                                                75
pâtes, riz et céréales                                71
biscuits sucrés                                       65
poissons cuits                                        63
pains et assimilés                                    56
produits à base de poissons et produits de la mer     56
fruits à coque et graines oléagineuses                52
pommes de terre et autres tuber

In [19]:
#Remplace tous les 'traces' en 0 et les '-' en 'Nan' dans les colonnes suivantes
for column in df_propre['Kcal/100 g']:
            df_propre['Kcal/100 g'] = df_propre['Kcal/100 g'].replace(['traces'], ['0'])
            df_propre['Kcal/100 g'] = df_propre['Kcal/100 g'].replace(['-'], value = np.nan, regex = True)            
            
for column in df_propre['Protéines/100 g']:
            df_propre['Protéines/100 g'] = df_propre['Protéines/100 g'].replace(['traces'], ['0'])
            df_propre['Protéines/100 g'] = df_propre['Protéines/100 g'].replace(['-'], value = np.nan, regex = True)

for column in df_propre['Glucides/100 g']:
            df_propre['Glucides/100 g'] = df_propre['Glucides/100 g'].replace(['traces'], ['0'])
            df_propre['Glucides/100 g'] = df_propre['Glucides/100 g'].replace(['-'], value = np.nan, regex = True)

for column in df_propre['Lipides/100 g']:
            df_propre['Lipides/100 g'] = df_propre['Lipides/100 g'].replace(['traces'], ['0'])
            df_propre['Lipides/100 g'] = df_propre['Lipides/100 g'].replace(['-'], value = np.nan, regex = True)

for column in df_propre['Sucres/100 g']:
            df_propre['Sucres/100 g'] = df_propre['Sucres/100 g'].replace(['traces'], ['0'])
            df_propre['Sucres/100 g'] = df_propre['Sucres/100 g'].replace(['-'], value = np.nan, regex = True)

for column in df_propre['AG saturés/100 g']:
            df_propre['AG saturés/100 g'] = df_propre['AG saturés/100 g'].replace(['traces'], ['0'])
            df_propre['AG saturés/100 g'] = df_propre['AG saturés/100 g'].replace(['-'], value = np.nan, regex = True)

In [20]:
#On affiche le nombre 'Nan' de chaque colonne
df_propre.isna().sum()

alim_grp_nom_fr        0
alim_ssgrp_nom_fr      0
alim_code              0
alim_nom_fr            0
Kcal/100 g           736
Protéines/100 g       16
Glucides/100 g        79
Lipides/100 g         16
Sucres/100 g         349
AG saturés/100 g     270
dtype: int64

In [21]:
#On compte le nombre de ligne avant de supprimer ceux qui ont une valeur 'Nan'
nombre_total_avant = df_propre['alim_grp_nom_fr'].count()
print(nombre_total_avant)

2605


In [22]:
#On supprime toutes les lignes qui ont une valeur 'Nan'
for column in df_propre:
            df_propre = df_propre.dropna()

In [23]:
#On vérifie
df_propre.isna().sum()

alim_grp_nom_fr      0
alim_ssgrp_nom_fr    0
alim_code            0
alim_nom_fr          0
Kcal/100 g           0
Protéines/100 g      0
Glucides/100 g       0
Lipides/100 g        0
Sucres/100 g         0
AG saturés/100 g     0
dtype: int64

In [24]:
#On recompte le nombre de lignes
nombre_total_apres = df_propre['alim_grp_nom_fr'].count()
print(nombre_total_apres)

1571


In [25]:
#On fait le calcul pour savoir le pourcentage de pertes
print(round(100 - (nombre_total_apres / nombre_total_avant) * 100 , 2), '% de perdu')

39.69 % de perdu


In [26]:
#On regarde les pertes de la colonne 'alim_ssgrp_nom_fr'
df_propre['alim_ssgrp_nom_fr'].value_counts()

légumes                                              132
charcuteries et assimilés                            126
viandes crues                                        124
fromages et assimilés                                105
poissons crus                                         95
produits laitiers frais et assimilés                  85
eaux                                                  69
boissons sans alcool                                  66
poissons cuits                                        61
fruits                                                59
viandes cuites                                        58
pâtes, riz et céréales                                56
pains et assimilés                                    50
pommes de terre et autres tubercules                  47
céréales de petit-déjeuner                            46
soupes                                                45
produits à base de poissons et produits de la mer     45
huiles et graisses végétales   

In [27]:
#On remplace les virgules par des points et on enelve '<' devant certaines valeurs
cols = ['Kcal/100 g', 'Protéines/100 g', 'Glucides/100 g', 'Lipides/100 g', 'Sucres/100 g', 'AG saturés/100 g']

for col in cols:
    df_propre[col] = df_propre[col].map(lambda x: str(x).replace(',', '.').lstrip('<')).astype(float)

Maintentant qu'on a fini de nettoyer les valeurs numériques, on va normaliser les valeurs textuelles.

### <font color = "blue">3.2- Valeur textuelle </font>

In [28]:
french_stopwords = set(stopwords.words('french'))
filtre_stopfr = lambda text: [token for token in text if token.lower() not in french_stopwords]

In [29]:
def scrapper(url):
    recipe = []
    page = requests.get(url)
    # https://www.marmiton.org/recettes/recette_sauce-puttanesca_14929.aspx
    tree = html.fromstring(page.content)
    products = tree.xpath("//div[contains(@class, 'MuiGrid-root')]")
    for product in products[1:]:
        recipe.append(product.text_content())
    return(recipe)

In [30]:
def normalize_text(text, stemmer=True):
    text = unaccented_string = unidecode.unidecode(str(text))    
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    words = filtre_stopfr( word_tokenize(text, language="french") )
    if stemmer:
        stemmer = FrenchStemmer()
        res = " ".join([stemmer.stem(w) for w in words])
    else:
        res = " ".join(words)
    return res

In [31]:
def word_in_column(df_propre, prod, col):
    mask = df_propre[col].str.contains(prod)
    return df_propre.loc[mask]

In [32]:
def list_match(name):
    l_match = []
    words_test = normalize_text(name).split(" ")
    for word in words_test:
        l_match.append(word_in_column(df_propre, word, 'alim_nom_fr'))
    res = pd.concat(l_match)
    df = pd.DataFrame(res)
    #res.reset_index()["index"].value_counts().iloc[0]
    return df

In [33]:
#Afficher la liste des index avec le nombre de fois que ca s'affiche
res.reset_index()["index"].value_counts()
#Afficher le premier et plus grand résultat
id_res = res.reset_index()["index"].value_counts().index[0]
#Afficher le nombre de fois que ca revient
res.reset_index()["index"].value_counts().iloc[0]

NameError: name 'res' is not defined

In [34]:
def best_res(res):
    id_res = res.reset_index()["index"].value_counts().index[0]
    print(df_propre.loc[id_res])
    return id_res;

In [35]:
def matcher(name):
    res = list_match(name)
    id_res = best_res(res)
    return (id_res)

In [36]:
def food_matcher(nom_aliment):
    prod = normalize_text(nom_aliment)
    products_fit = words_in_column(df_propre, prod, 'alim_nom_fr')
    print(f"size products {products_fit.shape[0]}")
    shorter_description_sample = products_fit["alim_nom_fr"].apply(len).sort_values()[:10].index
    print(shorter_description_sample)
    ten_shorter_des_match = df_propre.loc[shorter_description_sample]
    print(ten_shorter_des_match["alim_ssgrp_nom_fr"].value_counts())
    best_cat = ten_shorter_des_match["alim_ssgrp_nom_fr"].value_counts().index[0]
    if best_cat == "nan":
        best_cat = ten_shorter_des_match["alim_ssgrp_nom_fr"].value_counts().index[1]
    ten_shorter_des_match = ten_shorter_des_match.loc[ten_shorter_des_match["alim_ssgrp_nom_fr"]==best_cat]
    best_bet = ten_shorter_des_match.iloc[0]
    print(f"Best match : {best_bet['alim_nom_fr']} \nBest cat : {best_cat}")
    return best_bet

In [37]:
def list_to_tab(list_ing):
    tab = []
    for value in list_ing:
        u_mes = value.split(' ', 1)[0]
        if (u_mes is None):
            u_mes = 1
        attr = value.split(' ', 1)[1]
        temp = normalize_text(attr,False)
        tab.append([u_mes, temp])
    return (tab)

In [57]:
def nutr_calculator(df_ingr, quantity, unity):
    kcal = df_ingr["Kcal/100 g"]
    proteines = df_ingr["Protéines/100 g"]
    glucides = df_ingr["Glucides/100 g"]
    lipides = df_ingr["Lipides/100 g"]
    sucres = df_ingr["Sucres/100 g"]
    ag_satures = df_ingr["AG saturés/100 g"]
    
    q = int(quantity)
    
    print("kcal : " + str(kcal) + "\nprotéines : " + str(proteines) + "\nglucides : "+ str(glucides) + "\nlipides : "+ str(lipides) +  "\nsucres : "+ str(sucres) + "\nag_saturés : " + str(ag_satures))
    
    if (unity == 'g'):
        kcal = kcal * (q/100)
        proteines = proteines * (q/100)
        glucides = glucides * (q/100)
        lipides = lipides * (q/100)
        sucres = sucres * (q/100)
        ag_satures = ag_satures * (q/100)
        
    elif (unity == 'mg'):
        kcal = kcal * (q/1000)
        proteines = proteines * (q/1000)
        glucides = glucides * (q/1000)
        lipides = lipides * (q/1000)
        sucres = sucres * (q/1000)
        ag_satures = ag_satures * (q/1000)
        
    elif (unity == "cl"):
        kcal = kcal * (q/10)
        proteines = proteines * (q/10)
        glucides = glucides * (q/10)
        lipides = lipides * (q/10)
        sucres = sucres * (q/10)
        ag_satures = ag_satures * (q/10)
        
    elif (unity == "l"):
        kcal = kcal * (q*10)
        proteines = proteines * (q*10)
        glucides = glucides * (q*10)
        lipides = lipides * (q/10)
        sucres = sucres * (q*10)
        ag_satures = ag_satures * (q*10)
        
    elif (unity == "ml"):
        kcal = kcal * (q/100)
        proteines = proteines * (q/100)
        glucides = glucides * (q/100)
        lipides = lipides * (q/100)
        sucres = sucres * (q/100)
        ag_satures = ag_satures * (q/100)
        
    elif (unity == "c.à.s"):
        kcal = (kcal * (q/100)) * 1.5
        proteines = (proteines * (q/100)) * 1.5
        glucides = (glucides * (q/100)) * 1.5
        lipides = (lipides * (q/1000)) * 1.5
        sucres = (sucres * (q/100)) * 1.5
        ag_satures = (ag_satures * (q/100)) * 1.5
        
    #Le cas pour les oeufs  
    
    #elif (unity == 0 and df_ingr["Name"] == 1916):
        #kcal = (kcal * (q/100)) * 0.5
        #proteines = (proteines * (q/100)) * 0.5
        #glucides = (glucides * (q/100)) * 0.5
        #sucres = (sucres * (q/100)) * 0.5
        #ag_satures = (ag_satures * (q/100)) * 0.5

    else:
        kcal = 0
        proteines = 0
        glucides = 0
        lipides = 0
        sucres = 0
        ag_satures = 0
        
    tab = [kcal, proteines, glucides, lipides ,sucres, ag_satures]
    
    return (tab)

    

In [58]:
def food_calculator(list_ing):
    total_nutr = [0, 0, 0, 0, 0, 0]
    tmp = [0, 0, 0, 0, 0, 0]
    for value in list_ing:
        if (value[0] == ''):
            quantity = 0
            unity = 0          
        else :
            quantity = value[0].split('\xa0', 1)[0]
            if (len(value[0]) == 1):
                unity = 0
            else:
                unity = value[0].split('\xa0', 1)[1] 
        df_ingr = df_propre.loc[matcher(value[1])]        
         
        print("+++++++++")
        print("Quantite: " + str(quantity))
        print("Unity: " + str(unity))
        print("*********")
        tmp = nutr_calculator(df_ingr, quantity, unity)
        total_nutr = [b_elt + a_elt for a_elt, b_elt in zip(total_nutr, tmp)]
        print(tmp)
        print("+++++++++++++++++")
        
    print("\n\n\n\n\nTotal des Nutriments: " + str(total_nutr) + "\n\n\n\n")
    return(total_nutr)

In [72]:
def note(tab):
    
    somme_macro = tab[1] + tab[2] + tab[3]
    pourcentage_prot = (tab[1] / somme_macro) * 100
    pourcentage_glu = (tab[2] / somme_macro) * 100
    pourcentage_lip = (tab[3] / somme_macro) * 100
        
    #liste note element 0 = maintenir, 1 = perdre et 2 = masse 
    note = [20, 20 , 20]

    if (pourcentage_prot < 10 or pourcentage_prot > 40):
        note[0] = note[0] - 5
        note[1] = note[1] - 5
        note[2] = note[2] - 5

    if (pourcentage_prot >= 10 and pourcentage_prot <= 30):
        note[1] = note[1] - 5
        note[2] = note[2] - 5

    if (pourcentage_prot > 30 and pourcentage_prot <= 35):
        note[0] = note[0] - 5
        note[2] = note[2] - 5

    if (pourcentage_prot > 35 and pourcentage_prot <= 40):
        note[0] = note[0] - 5
        note[1] = note[1] - 5

    if (pourcentage_glu < 35 or pourcentage_glu > 65):
        note[0] = note[0] - 5
        note[1] = note[1] - 5
        note[2] = note[2] - 5

    if (pourcentage_glu >= 35 and pourcentage_glu <= 45):
        note[0] = note[0] - 5
        note[2] = note[2] - 5

    if (pourcentage_glu > 45 and pourcentage_glu <= 50):
        note[0] = note[0] - 5
        note[1] = note[1] - 5

    if (pourcentage_glu > 50 and pourcentage_glu <= 65):
        note[1] = note[1] - 5
        note[2] = note[2] - 5

    if (pourcentage_lip < 10 or pourcentage_lip > 35):
        note[0] = note[0] - 5
        note[1] = note[1] - 5
        note[2] = note[2] - 5

    if (pourcentage_lip >= 10 and pourcentage_lip < 20):
        note[0] = note[0] - 5
        note[1] = note[1] - 5

    if (pourcentage_lip >= 20 and pourcentage_lip < 25):
        note[0] = note[0] - 5
        note[2] = note[2] - 5

    if (pourcentage_lip >= 25 and pourcentage_lip < 35):
        note[1] = note[1] - 5
        note[2] = note[2] - 5
        
    print(note)
    return(note)

In [68]:
def food_checker_main(url):
    recipe = scrapper(url)
    df_propre["alim_nom_fr"] = df_propre["alim_nom_fr"].apply(normalize_text)
    df_propre["alim_ssgrp_nom_fr"] = df_propre["alim_ssgrp_nom_fr"].apply(normalize_text)
    ingr_list = list_to_tab(recipe)
    
    print("Liste des ingrédients: \n\n")
    print(ingr_list)
    print("Calcul des nutriments: \n\n")
    tab_nutriments = food_calculator(ingr_list)
    note(tab_nutriments)

In [73]:
food_checker_main("https://www.marmiton.org/recettes/recette_sauce-bolognaise-ragu-bolognese-recette-officielle_95684.aspx")

Liste des ingrédients: 


[['2\xa0c.à.s', 'huile olive'], ['10\xa0cl', 'vin rouge Sangiovese sec'], ['20\xa0cl', 'bouillon'], ['500\xa0g', 'puree tomate passata'], ['50\xa0g', 'oignon hache'], ['50\xa0g', 'carotte jaune hache'], ['', 'poivre'], ['', 'sel'], ['500\xa0g', 'pates'], ['300\xa0g', 'boeuf maigre hache'], ['100\xa0g', 'lard doux'], ['50\xa0g', 'celeri branche hache']]
Calcul des nutriments: 


alim_grp_nom_fr      viandes, œufs, poissons et assimilés
alim_ssgrp_nom_fr       produit a bas poisson produit mer
alim_code                                           26040
alim_nom_fr              sardin a huil oliv appert egoutt
Kcal/100 g                                          202.0
Protéines/100 g                                      24.3
Glucides/100 g                                       1.06
Lipides/100 g                                        11.2
Sucres/100 g                                          0.0
AG saturés/100 g                                     3.13
Name: 1875, d

maintenir son poids 45 à 65% de glucides, 20 à 35% de lipides et 10 à 35% de protéines.

perdre du poids, un ratio de 35% de glucides, 30% de lipides et de 35% de protéines.

masse musculaire 50% de glucides, 10% de lipides et 40% de protéines




Par exemple, si vous décidez de suivre un régime cétogène pour maigrir, le ratio sera le suivant : 10% de glucides, 50% de lipides et 40% de protéines.

régime hyperprotéiné, le ratio sera de 10% de glucides, 25% de lipides et 65% de protéines.